In [52]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone 
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [ ]:
import os


In [55]:
PINECONE_API_KEY = "9147740b-d72f-4b51-803e-447af9f00836"
PINECONE_API_ENV = "chatkey"

In [ ]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["PINECONE_API_ENV"] = PINECONE_API_ENV

In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [10]:
#extracted_data = load_pdf("data/")
extracted_data = load_pdf(r"C:\Users\smsub\Subahani\Study\Generative AI\chatbotLlama2\GenAI-End-to-End-Medical-chatbot-using-LLAMA2\data")

In [12]:
extracted_data[0]

Document(page_content='', metadata={'source': 'C:\\Users\\smsub\\Subahani\\Study\\Generative AI\\chatbotLlama2\\GenAI-End-to-End-Medical-chatbot-using-LLAMA2\\data\\Medical_book.pdf', 'page': 0})

In [13]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [14]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [16]:
text_chunks[0].page_content

'TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION'

In [17]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [20]:
embeddings = download_hugging_face_embeddings()

In [21]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [22]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [23]:
query_result

[-0.034477222710847855,
 0.031023189425468445,
 0.00673493230715394,
 0.02610897831618786,
 -0.03936203941702843,
 -0.16030248999595642,
 0.06692398339509964,
 -0.006441492587327957,
 -0.04745054617524147,
 0.01475894171744585,
 0.07087540626525879,
 0.05552757903933525,
 0.019193286076188087,
 -0.026251375675201416,
 -0.010109512135386467,
 -0.026940539479255676,
 0.022307513281702995,
 -0.022226620465517044,
 -0.14969266951084137,
 -0.017493048682808876,
 0.00767626753076911,
 0.0543522946536541,
 0.003254437353461981,
 0.03172598034143448,
 -0.0846213847398758,
 -0.029406018555164337,
 0.05159567669034004,
 0.048124030232429504,
 -0.0033148014917969704,
 -0.05827920511364937,
 0.04196932911872864,
 0.022210698574781418,
 0.12818889319896698,
 -0.022338908165693283,
 -0.011656295508146286,
 0.06292832642793655,
 -0.03287626430392265,
 -0.09122607111930847,
 -0.03117542713880539,
 0.052699606865644455,
 0.0470348484814167,
 -0.08420299738645554,
 -0.030056146904826164,
 -0.02074474096

In [56]:
#Initializing the Pinecone
pinecone.Pinecone(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="myindex"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

PineconeConfigurationError: You haven't specified an Api-Key.